In [1]:
#Allow Notebook to Import from Comp_Chem_Package
import sys
if("win" in sys.platform):
    sys.path.append("..\\..\\Backend")
else: 
    sys.path.append("../../Backend")
    
from nistScraper import getDiatomicConstants 
from morsePotential import morsePotential
from howPotential import howPotential
from wavefunction import wavefunction
from operators import HOperator
from schrodinger import schrod
from basisSet import basisSet
from compChemGlobal import *
from how import how
from rkr import rkr

## Harmonic Oscillator Wavefunction
<br>
The harmonic oscillator wavefunction arises from solving the Schrödinger for the harmonic oscillator. <br>
Answer the following questions on the below regarding the harmonic oscillator: 

In [2]:
from diatomicConstants import buildDiatomicConstants
diatomicSymbol = widgets.Text(value = "CO")
u = widgets.Dropdown(
    options = ["Normal", "Times 2", "Divide 2"], 
    description = "μ", 
    value = "Normal"
)
w = widgets.Dropdown(
    options = ["Normal", "Times 2", "Divide 2"], 
    description = "ω", 
    value = "Normal"
)
diatomic = widgets.HBox([widgets.Label("Diatomic Symbol"), diatomicSymbol, u, w])
calculate = startButton = widgets.ToggleButton(
    value = False, 
    description = "Update Solution",
    button_style = "info"
)
info = widgets.HTML("")
graph = widgets.VBox()
out = widgets.Output()

def buildSolution(dc, name = ""):
    pes =  howPotential(dc)
    basis = basisSet(dc, how)
    sol = schrod()
    sol.graphTitle = basis.diatomicConstants["name"] + " Harmonic Oscilator Schrödinger Solution"
    for index, bf in enumerate(basis):
        #eigen value analytical equation derived from hyperphysics website for harmonic osciallator
        
        ev = (index + 0.5) * dc["w"]
        evec = [1 if i == index else 0 for i in range(basis.size)]
        wf = wavefunction(evec, ev, basis, index).scale(500)
        wf2 = wavefunction(evec, ev, basis, index, squared=True).scale(500)
        wf.setGraphVariables(yEqualsCutoff = ev)
        wf2.setGraphVariables(yEqualsCutoff = ev)
        
        
        sol.addGraphableObject(wf)
        sol.addGraphableObject(wf2)
        
    sol.addGraphableObject(pes)
    sol.scaleFactor = 500
    sol.start = dc["re"] - 0.4
    sol.end = dc["re"] + 0.4
    sol.pesLocations.append(len(sol.graphableObjects) - 1)
    
    return sol
 
#-------------------------------------------------------------------------------------
    
def howPESUpdate():
    dc = getDiatomicConstants(diatomicSymbol.value)
    if(dc == False):
        info.value = "'" + diatomicSymbol.value + "' is not a valid diatomic molecule." 
        return
    info.value = "Calculating solution for " + diatomicSymbol.value + "." 
    
    sol = buildSolution(dc)
    name = ""
    #update diatomic constants
    for widget, index in [(u, "u"), (w, "w")]:
        if(widget.value == "Times 2"):
            dc[index] *= 2
            name += ", " + widget.description + " " + widget.value
        elif(widget.value == "Divide 2"):
            dc[index] /= 2
            name += ", " + widget.description + " " + widget.value
    
    if(name != ""):
        sol.combineSolutions(buildSolution(dc, name), name)

    graph.children = ( (sol.graph(getGraph = True), diatomic, calculate, info) )
    
    out.clear_output()
    with out: 
        display(graph)
    
    info.value = "Calculation Completed"

updateWrapper = lambda change :  howPESUpdate()

calculate.observe(updateWrapper, names="value")

howPESUpdate()  
display(out)

Output()

<br><br>
Answer the below qustions using the interactive graph of the v = 0, 1 and 2 wavefunctions.

<br><br>
Using the graph above, change the mode option to "Probability Distribution", and answer the following questions in units of angstroms. 

<br>
The following questions will revolve around comparing a model of a diatomic molecule using a harmonic potential energy surface, and an anharmonic poetential energy surface. Specifically, the harmonic potential will be the haromonic oscillator presented above, while the anharmonic potential will be the anharmonic morse potential. 

In [3]:
diatomicSymbol = widgets.Text(value = "CO")

diatomic = widgets.HBox([widgets.Label("Diatomic Symbol"), diatomicSymbol])
calculate = startButton = widgets.ToggleButton(
    value = False, 
    description = "Update Solution",
    button_style = "info"
)
info = widgets.HTML("")
graph = widgets.VBox()
out2 = widgets.Output()

def howPESUpdate():
    dc = getDiatomicConstants(diatomicSymbol.value)
    if(dc == False):
        info.value = "'" + diatomicSymbol.value + "' is not a valid diatomic molecule." 
        return
    info.value = "Calculating solution for " + diatomicSymbol.value + "." 
    
    #calculate solution for anharmonic system
    basis = basisSet(dc, how, size = 4)
    anharmonicPES = morsePotential(dc).fit(rkr(dc).compute())
    sol = schrod(HOperator(basis, anharmonicPES), basis, pes = anharmonicPES)
        
    pes =  howPotential(dc)
    sol.graphTitle = basis.diatomicConstants["name"] + " Harmonic Oscilator Schrödinger Solution"
    for index, bf in enumerate(basis):
        #eigen value analytical equation derived from hyperphysics website for harmonic osciallator
        
        ev = (index + 0.5) * dc["w"]
        evec = [1 if i == index else 0 for i in range(basis.size)]
        wf = wavefunction(evec, ev, basis, index).scale(sol.scaleFactor)
        wf2 = wavefunction(evec, ev, basis, index, squared=True).scale(sol.scaleFactor)
        wf.setGraphVariables(yEqualsCutoff = ev, graphTitle = wf.graphTitle + " Harmonic")
        wf2.setGraphVariables(yEqualsCutoff = ev, graphTitle = wf2.graphTitle + " Harmonic")
        
        sol.addGraphableObject(wf)
        sol.addGraphableObject(wf2)
        
    sol.addGraphableObject(pes)
    sol.pesLocations.append(len(sol.graphableObjects) - 1)
    sol.start = dc["re"] - 0.4
    sol.end = dc["re"] + 0.4
 
    graph.children = ( (sol.graph(getGraph = True), diatomic, calculate, info) )
    info.value = "Calculation Completed"
    
    out2.clear_output()
    with out2: 
        display(graph)

updateWrapper = lambda change :  howPESUpdate()
calculate.observe(updateWrapper, names="value")

howPESUpdate()  
display(out2)
#basis of 100, precalculate and save for later

Output()